In [1]:
import struct
import datetime as dt
import glob
import os
from collections import namedtuple
import matplotlib.image as mpimg
import cv2

import numpy as np
# import pykitti.utils as utils

In [2]:
# fileName = '/Users/GF/Downloads/data_road_velodyne/testing/velodyne/um_000000.bin'
# with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
# struct.unpack("iiiii", fileContent[:20])

In [3]:
def get_velo_scans(velo_file):
    """read velodyne [x,y,z,reflectance] scan data from binary files."""
    """parse velodyne binary files into arrays."""
    scan = np.fromfile(velo_file, dtype='float32')
    return scan.reshape((-1, 4))
        
# def velo(data_path):
#         """Generator to read velodyne [x,y,z,reflectance] scan data from binary files."""
#         # Find all the Velodyne files
#         velo_path = os.path.join(
#             data_path, 'velodyne_points', 'data', '*.bin')
#         velo_files = sorted(glob.glob(velo_path))

#         # Subselect the chosen range of frames, if any
# #         if self.frames is not None:
# #             velo_files = [velo_files[i] for i in self.frames]

#         # Return a vgenerator yielding Velodyne scans.
#         # Each scan is a Nx4 array of [x,y,z,reflectance]
#         return get_velo_scans(velo_files)

In [41]:
#filename = '/Users/GF/Downloads/data_road_velodyne-2/training/velodyne/um_000003.bin'
filename = '/Users/Mike/Desktop/data_road_velodyne/training/velodyne/um_000002.bin'
arr = get_velo_scans(filename)
new_arr = np.full((400,400,4),(0,0,0,255))
for point in arr:
    if(point[3] > 0 and point[0] > 0 and point[0] <= 19.95 and abs(point[1]) < 9.95 and point[2] < 0):
        x_coord = int(round(point[1]*20)+200)
        y_coord = int(round(point[0]*20))
        new_arr[399-y_coord][x_coord] = (0,point[3]*255,0,255)
cv2.imwrite("out.png", new_arr)
        
# scan = np.fromfile(filename, dtype='uint8')
# cv2.imwrite('scan.png', scan[:640000].reshape(64,10000))

True